In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


from selenium import webdriver #selenium 4.20.0
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import WebDriverException

from webdriver_manager.chrome import ChromeDriverManager
 # version 4.0.1

In [2]:
options = webdriver.ChromeOptions()
options.set_capability(
    "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
)

# Make sure you already have Chrome installed
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [3]:
driver.set_page_load_timeout(30)

try:
    driver.get("https://www.sofascore.com/api/v1/player/923136/unique-tournament/1015/season/52519/statistics/overall")
except:
    pass


driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [4]:
# extract requests from logs
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

In [5]:
desired_keywords = ['statistics']

for log in logs:
    # Check if the log contains the 'method' key
    if 'method' in log:
        # Check if the log method is 'Network.responseReceived'
        if log["method"] == "Network.responseReceived":
            # Extract request ID and URL
            request_id = log["params"]["requestId"]
            url = log["params"]["response"]["url"]
            # Check if the URL contains any of the desired keywords
            if any(keyword in url for keyword in desired_keywords):
                # Print the request ID and URL
                print(f"Request ID: {request_id}, URL: {url}")
                # # Try to retrieve the response body using the request ID
                try:
                    response = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': request_id})
                    stats = response['body']
                    if stats:
                        print("Response body successfully retrieved:")
                        print(stats)
                        break
                    else:
                        print("Response body is empty. Trying next request ID...")
                except Exception as e:
                    print(f"Error occurred while retrieving response for request ID {request_id}: {e}")
                    continue

stats


Request ID: 8191593DD26BF81F67CED16A2588895B, URL: https://www.sofascore.com/api/v1/player/923136/unique-tournament/1015/season/52519/statistics/overall
Response body successfully retrieved:
{"statistics":{"rating":6.8791666666667,"goals":1,"assists":1,"goalsAssistsSum":2,"accuratePasses":684,"inaccuratePasses":92,"totalPasses":776,"accuratePassesPercentage":88.14,"accurateFinalThirdPasses":126,"keyPasses":11,"successfulDribbles":9,"successfulDribblesPercentage":75,"interceptions":54,"yellowCards":8,"directRedCards":0,"redCards":1,"accurateCrosses":7,"accurateCrossesPercentage":41.18,"totalShots":13,"shotsOnTarget":3,"shotsOffTarget":10,"aerialDuelsWon":11,"aerialDuelsWonPercentage":57.89,"totalDuelsWon":76,"totalDuelsWonPercentage":32.9,"minutesPlayed":1495,"goalConversionPercentage":7.692,"penaltiesTaken":0,"penaltyGoals":0,"shotFromSetPiece":2,"accurateLongBalls":63,"accurateLongBallsPercentage":63,"clearances":21,"errorLeadToShot":4,"wasFouled":22,"fouls":40,"dribbledPast":46,"offs

'{"statistics":{"rating":6.8791666666667,"goals":1,"assists":1,"goalsAssistsSum":2,"accuratePasses":684,"inaccuratePasses":92,"totalPasses":776,"accuratePassesPercentage":88.14,"accurateFinalThirdPasses":126,"keyPasses":11,"successfulDribbles":9,"successfulDribblesPercentage":75,"interceptions":54,"yellowCards":8,"directRedCards":0,"redCards":1,"accurateCrosses":7,"accurateCrossesPercentage":41.18,"totalShots":13,"shotsOnTarget":3,"shotsOffTarget":10,"aerialDuelsWon":11,"aerialDuelsWonPercentage":57.89,"totalDuelsWon":76,"totalDuelsWonPercentage":32.9,"minutesPlayed":1495,"goalConversionPercentage":7.692,"penaltiesTaken":0,"penaltyGoals":0,"shotFromSetPiece":2,"accurateLongBalls":63,"accurateLongBallsPercentage":63,"clearances":21,"errorLeadToShot":4,"wasFouled":22,"fouls":40,"dribbledPast":46,"offsides":0,"blockedShots":5,"passToAssist":2,"saves":0,"cleanSheet":0,"crossesNotClaimed":0,"matchesStarted":17,"penaltyConversion":0,"totalCross":17,"duelLost":155,"aerialLost":8,"attemptPenal

In [6]:
data = json.loads(stats)
df = pd.json_normalize(data)
df

,statistics.rating,statistics.goals,statistics.assists,statistics.goalsAssistsSum,statistics.accuratePasses,statistics.inaccuratePasses,statistics.totalPasses,statistics.accuratePassesPercentage,statistics.accurateFinalThirdPasses,statistics.keyPasses,...,team.nameCode,team.disabled,team.national,team.type,team.id,team.teamColors.primary,team.teamColors.secondary,team.teamColors.text,team.fieldTranslations.nameTranslation.ar,team.fieldTranslations.nameTranslation.ru
0,6.879167,1,1,2,684,92,776,88.14,126,11,...,PEJ,False,False,0,64295,#ff0000,#ffffff,#ffffff,بيرسجة جاكرتا,Персия Джакарта


In [7]:
df.to_csv('statistic_syahrianAbimanyu.csv', index=False)